In [7]:
import arff
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.model_selection import cross_validate

## training model
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [5]:
def get_recal_1(y_true, y_pred):
    precision, recall, fscore, support = score(y_true, y_pred)
    print(classification_report(y_true, y_pred))
    return(recall[1])

In [13]:
nltk.download('stopwords')
nltk.download('rslp')
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()
nltk.download('punkt')


def clean_text(txt):
    text = ''
    for w in nltk.word_tokenize(txt):
        if w not in stopwords:
            text = text + stemmer.stem(w) + ' '
    return text.strip()

[nltk_data] Downloading package stopwords to /home/marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/marco/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Fase 1

In [10]:
data = arff.load(open('../OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()

cl =  Pipeline([
        ('tfidf',TfidfVectorizer()),
        ('clf', DecisionTreeClassifier()),
        ])

scores = cross_validate(cl, X, y,
                        cv=5, return_train_score=False, scoring=make_scorer(get_recal_1))

              precision    recall  f1-score   support

           0       0.85      0.88      0.87       167
           1       0.44      0.39      0.42        41

   micro avg       0.78      0.78      0.78       208
   macro avg       0.65      0.64      0.64       208
weighted avg       0.77      0.78      0.78       208

              precision    recall  f1-score   support

           0       0.86      0.75      0.80       166
           1       0.32      0.49      0.39        41

   micro avg       0.70      0.70      0.70       207
   macro avg       0.59      0.62      0.59       207
weighted avg       0.75      0.70      0.72       207

              precision    recall  f1-score   support

           0       0.85      0.73      0.79       166
           1       0.30      0.47      0.37        40

   micro avg       0.68      0.68      0.68       206
   macro avg       0.57      0.60      0.58       206
weighted avg       0.74      0.68      0.70       206

              preci

## Fase 2

In [14]:
data = arff.load(open('../OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()

## Cleaning text before

for i in range(len(X)):
    n_txt = clean_text(X[i])
    X[i] = n_txt
    
cl =  Pipeline([
        ('tfidf',TfidfVectorizer()),
        ('clf', DecisionTreeClassifier()),
        ])

scores = cross_validate(cl, X, y,
                        cv=5, return_train_score=False, scoring=make_scorer(get_recal_1))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       167
           1       0.61      0.41      0.49        41

   micro avg       0.83      0.83      0.83       208
   macro avg       0.74      0.67      0.70       208
weighted avg       0.82      0.83      0.82       208

              precision    recall  f1-score   support

           0       0.84      0.87      0.86       166
           1       0.40      0.34      0.37        41

   micro avg       0.77      0.77      0.77       207
   macro avg       0.62      0.61      0.61       207
weighted avg       0.76      0.77      0.76       207

              precision    recall  f1-score   support

           0       0.86      0.89      0.87       166
           1       0.45      0.38      0.41        40

   micro avg       0.79      0.79      0.79       206
   macro avg       0.66      0.63      0.64       206
weighted avg       0.78      0.79      0.78       206

              preci

## Fase 3

In [ ]:
data = arff.load(open('../OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()

## Cleaning text before

for i in range(len(X)):
    n_txt = clean_text(X[i])
    X[i] = n_txt
    
cl =  Pipeline([
        ('tfidf',TfidfVectorizer()),
        ('clf', DecisionTreeClassifier()),
        ])
    
parameters = {'tfidf__ngram_range': [(1,1), (1,2), (1,3), (1,4)],
              'clf__criterion': ('gini', 'entropy'),
              'clf__class_weight': ({0: 1, 1: 1}, {0: 1, 1: 2}, {0: 1, 1: 3}, {0: 1, 1: 4}, {0: 2, 1: 1},
                                    {0: 2, 1: 1}, {0: 3, 1: 1}, {0: 4, 1: 1}),
              'clf__min_samples_split': (2, 3, 4, 5, 6, 7, 8)
             }

gs_clf = GridSearchCV(cl, parameters, cv=5, iid=False, n_jobs=-1, scoring='precision')

gs_clf.fit(X, y)
print(gs_clf.best_score_)
print(gs_clf.best_params_)
    